In [89]:
%reset -f
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_curve, auc
from torch.utils.data import random_split, DataLoader


from customDatasets.audioDataset import AudioDataset


In [90]:
# free gpu
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [91]:
class ConvolutionalAE(nn.Module):
    def __init__(self, encoding_dim):
        super(ConvolutionalAE, self).__init__()
        self.encoding_dim = encoding_dim
        self.encoder = nn.Sequential(
            # (320, 128)
            nn.Conv2d(1, 32, kernel_size=5, stride=(1,2), padding=(2,2)),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # (320, 64)
            nn.Conv2d(32, 64, kernel_size=5, stride=(1,2), padding=(2,2)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # (320, 32)
            nn.Conv2d(64, 128, kernel_size=5, stride=(2,2), padding=(2,2)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # (160, 16)
            nn.Conv2d(128, 256, kernel_size=3, stride=(2,2), padding=(1,1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            # (80, 8)
            nn.Conv2d(256, 512, kernel_size=3, stride=(2,2), padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # (40, 4, 512)
        )
               
        # inflates the latent space to the shape of the last layer of the encoder
        self.fc=nn.Sequential(
            nn.Linear(512*40*4, self.encoding_dim),
            nn.ReLU(),
            nn.Linear(self.encoding_dim, 512*40*4),
            nn.ReLU()
        )
        
        self.decoder=nn.Sequential(
            # (512, 40, 4)
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=(2,2), padding=(1,1),output_padding=(1,1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            # (256, 80, 8)
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=(2,2), padding=(1,1),output_padding=(1,1)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # (128, 160, 16)
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=(2,2), padding=(1,1),output_padding=(1,1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # (64, 320, 32)
            nn.ConvTranspose2d(64, 32, kernel_size=5, stride=(1,2), padding=(2,2),output_padding=(0,1)),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # (32, 320, 64)
            nn.ConvTranspose2d(32, 1, kernel_size=5, stride=(1,2), padding=(2,2),output_padding=(0,1)),
            # (1, 320, 128)
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        encoded = encoded.view(-1, 512*40*4)
        x=self.fc(encoded)
        x = x.view(-1, 512, 40, 4)
        decoded = self.decoder(x)
        
        return decoded

In [92]:
def train_model(model, train_dl, val_dl, test_dl, criterion, optimizer, device, epochs=5, step_size=5):
    lr_scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)
    for epoch in range(epochs):
        train_losses = []
        val_losses = []

        for inputs, labels in train_dl:
            model.train()
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, inputs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        lr_scheduler.step()
        print(f'Epoch[{epoch + 1}/{epochs}], Train loss: {np.average(train_losses): .4f}')
        
        
        for inputs, labels in val_dl:
            model.eval()
            with torch.no_grad():
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, inputs)
                val_losses.append(loss.item())
        print(f'Epoch[{epoch + 1}/{epochs}], Val loss: {np.average(val_losses): .4f}')
 
        scores = []
        full_labels = []
        for inputs, labels in test_dl:
            model.eval()
            with torch.no_grad():
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                mse = torch.sum((outputs-inputs)**2,dim=(1,2,3))/(inputs.shape[1]*inputs.shape[2]*inputs.shape[3])            
                scores.append(mse)
                full_labels.append(labels)
        
        full_labels = torch.cat([label for label in full_labels])
        scores = torch.cat([score for score in scores])
        fpr, tpr, _ = roc_curve(full_labels.cpu().detach(), scores.cpu().detach(), pos_label=0)
        roc_auc = auc(fpr, tpr)
        print(roc_auc)
        
    return np.average(train_losses),np.average(val_losses),roc_auc

In [93]:
def set_seed(seed = 42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

CONFIG = {
    "seed": 42,
    "epochs": 10,
    "num_classes": 2,
    "learning_rate": 0.01,
    "train_batch_size": 32,
    "val_batch_size": 16,
    "test_batch_size": 128,
    "criterion": nn.MSELoss(),
    "device":
        torch.device(
            "cuda:0" if torch.cuda.is_available()
            else "mps" if torch.backends.mps.is_available()
            else "cpu"
        )
}

set_seed(CONFIG['seed'])

data_path = "./data/train/"
data_path_test = "./data/test/"


meta_train_df = pd.read_csv("./data/train.csv")
meta_test_df = pd.read_csv("./data/test.csv")

train_df = meta_train_df[['filename', 'is_normal', 'machine_id']]
train_ds = AudioDataset(train_df, data_path)
test_df = meta_test_df[['filename', 'is_normal', 'machine_id']]
test_ds = AudioDataset(test_df, data_path_test)

num_items = len(train_ds)
num_train = int(0.8 * num_items)
num_val = num_items-num_train

train_ds, val_ds = random_split(train_ds, [num_train, num_val])
test_ds = test_ds


train_dl = DataLoader(train_ds, batch_size=CONFIG['train_batch_size'], shuffle=True)
val_dl = DataLoader(val_ds, batch_size=CONFIG['val_batch_size'], shuffle=False)
test_dl = DataLoader(test_ds, batch_size=CONFIG["test_batch_size"], shuffle=False)

In [94]:
model = ConvolutionalAE(encoding_dim=128)
model = model.to(CONFIG["device"])
optimizer = optim.Adam(model.parameters(), lr=CONFIG["learning_rate"])
for batch in train_dl:
    inputs, labels = batch
    print(inputs.shape)
    inputs = inputs.to(CONFIG["device"])
    outputs = model(inputs)
    print(outputs.shape)
    break

torch.Size([32, 1, 320, 128])
torch.Size([32, 1, 320, 128])


In [95]:
# compute metrics
inputs_cat=[]
for inputs, labels in train_dl:
    inputs_cat.append(inputs)
inputs_cat = torch.cat([input for input in inputs_cat])
print(inputs_cat.shape)

torch.Size([1896, 1, 320, 128])


In [96]:
# compute the min and max value for each frequency of the batch_sizexchannelxtimexfrequecy
min = torch.min(inputs_cat, dim=0).values
max = torch.max(inputs_cat, dim=0).values
print(max.shape)
print(min.shape)
train_ds.min = min
train_ds.max = max
test_ds.min = min
test_ds.max = max
val_ds.min = min
val_ds.max = max

torch.Size([1, 320, 128])
torch.Size([1, 320, 128])


In [97]:
measures = []
for emb_space_size in [32, 64, 128, 256, 512]:
    model = ConvolutionalAE(encoding_dim=emb_space_size)
    model = model.to(CONFIG["device"])
    optimizer = optim.Adam(model.parameters(), lr=CONFIG["learning_rate"])
    measures.append(train_model(model, train_dl, val_dl, test_dl, CONFIG["criterion"], optimizer, CONFIG["device"], CONFIG["epochs"]))
for emb_space_size, measure in zip([32, 64, 128, 256, 512], measures):
    print(f"Emb space size: {emb_space_size}, Train loss: {measure[0]}, Val loss: {measure[1]}, ROC AUC: {measure[2]}")

Epoch[1/10], Train loss:  41.2674
Epoch[1/10], Val loss:  21.4426
0.8055035372451103
Epoch[2/10], Train loss:  20.8006
Epoch[2/10], Val loss:  23.4689
0.8045900957136912
Epoch[3/10], Train loss:  20.1312
Epoch[3/10], Val loss:  20.5690
0.80867873491469
Epoch[4/10], Train loss:  19.3681
Epoch[4/10], Val loss:  26.0973
0.8094881398252185
Epoch[5/10], Train loss:  18.9961
Epoch[5/10], Val loss:  18.4890
0.8039741989180191
Epoch[6/10], Train loss:  18.6282
Epoch[6/10], Val loss:  18.3041
0.8074531835205994
Epoch[7/10], Train loss:  18.4152
Epoch[7/10], Val loss:  18.1432
0.8082188930503538
Epoch[8/10], Train loss:  18.2431
Epoch[8/10], Val loss:  18.1522
0.8064086558468581
Epoch[9/10], Train loss:  18.2286
Epoch[9/10], Val loss:  18.0469
0.8114024136496047
Epoch[10/10], Train loss:  18.1372
Epoch[10/10], Val loss:  18.1185
0.8083062838119017
Epoch[1/10], Train loss:  36.9517
Epoch[1/10], Val loss:  23.6796
0.8081960049937578
Epoch[2/10], Train loss:  21.1943
Epoch[2/10], Val loss:  20.5794

In [99]:
# take the best one and train it for more epochs
emb_space_measures=[32, 64, 128, 256, 512]
model = ConvolutionalAE(encoding_dim=emb_space_measures[np.argmax([measure[2] for measure in measures])])
model = model.to(CONFIG["device"])

Epoch[1/50], Train loss:  108.8960
Epoch[1/50], Val loss:  30.5281
0.8054806491885143
Epoch[2/50], Train loss:  23.0877
Epoch[2/50], Val loss:  20.7735
0.8127590511860175
Epoch[3/50], Train loss:  19.4100
Epoch[3/50], Val loss:  18.4904
0.7987702871410737
Epoch[4/50], Train loss:  18.4952
Epoch[4/50], Val loss:  17.5310
0.8085518102372036
Epoch[5/50], Train loss:  17.3512
Epoch[5/50], Val loss:  17.4059
0.8068705784436122
Epoch[6/50], Train loss:  16.4580
Epoch[6/50], Val loss:  16.1948
0.8077153558052435
Epoch[7/50], Train loss:  15.8361
Epoch[7/50], Val loss:  16.6327
0.8123741156887223
Epoch[8/50], Train loss:  15.2831
Epoch[8/50], Val loss:  15.4545
0.8074157303370786
Epoch[9/50], Train loss:  14.7621
Epoch[9/50], Val loss:  14.4026
0.8030316271327507
Epoch[10/50], Train loss:  14.6060
Epoch[10/50], Val loss:  14.6483
0.8075260091552227
Epoch[11/50], Train loss:  14.4223
Epoch[11/50], Val loss:  15.0697
0.803899292550978
Epoch[12/50], Train loss:  14.1682
Epoch[12/50], Val loss:  1

(10.602347278594971, 10.813386599222818, 0.8083270911360799)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=+0.0001)
#save weights into weights/weights.pth
torch.save(model.state_dict(), "./weights/weights.pth")
train_model(model, train_dl, val_dl, test_dl, CONFIG["criterion"], optimizer, CONFIG["device"], 50, 25)